### Importing useful libraries

In [1]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.layers import AveragePooling2D, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
import tensorflow as tf

### Preparing our Data

In [2]:
import cv2
DATA_PATH = "rpsls_data" # Path to folder containing data

shape_to_label = {'paper':np.array([1.,0.,0.,0.,0.]),'rock':np.array([0.,1.,0.,0.,0.]),'lizard':np.array([0.,0.,1.,0.,0.]),'spock':np.array([0.,0.,0.,1.,0.]),'scissors':np.array([0.,0.,0.,0.,1.])}
arr_to_shape = {np.argmax(shape_to_label[x]):x for x in shape_to_label.keys()}

imgData = list()
labels = list()

for dr in os.listdir(DATA_PATH):
    if dr not in ['paper','rock','lizard','spock','scissors']:
        continue
    print(dr)
    lb = shape_to_label[dr]
    for pic in os.listdir(os.path.join(DATA_PATH,dr)):
        path = os.path.join(DATA_PATH,dr+'/'+pic)
        image=load_img(path, target_size=(300, 300))
        image=img_to_array(image)
        imgData.append([image,lb])
        imgData.append([cv2.flip(image, 1),lb])
        #print(path)

np.random.shuffle(imgData)

imgData,labels = zip(*imgData)

imgData = np.array(imgData)
labels = np.array(labels)

#print(labels)

lizard
paper
rock
scissors
spock


In [3]:
len(labels)

60

In [4]:
labels

array([[0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1.

## Now let's prepare our model and train it

In [5]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [6]:
checkpoint = ModelCheckpoint(
    'model_rpsls.h5', 
    monitor='val_acc', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=True,
    mode='auto'
)

es = EarlyStopping(patience = 3)

In [7]:
model=tf.keras.models.Sequential([tf.keras.layers.Conv2D(100, (3,3), activation='relu', input_shape=(300, 300, 3)), 
                                 tf.keras.layers.MaxPooling2D(),
                                 tf.keras.layers.Conv2D(100, (3,3), activation='relu'),
                                 tf.keras.layers.MaxPooling2D(),
                                 tf.keras.layers.Flatten(),
                                 tf.keras.layers.Dense(50, activation='relu'),
                                 tf.keras.layers.Dense(5, activation='softmax')])

In [8]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [9]:
history = model.fit(
    x=imgData,
    y=labels,
    batch_size = 16,
    epochs=8,
    callbacks=[checkpoint,es],
    validation_split=0.2
)

Train on 48 samples, validate on 12 samples
Epoch 1/8
32/48 [===================>..........] - ETA: 6s - loss: 2697.3025 - acc: 0.1562
Epoch 00001: val_acc improved from -inf to 0.25000, saving model to model_rpsls.h5
48/48 [==============================] - 21s 433ms/sample - loss: 2275.9943 - acc: 0.1250 - val_loss: 1408.8353 - val_acc: 0.2500
Epoch 2/8
32/48 [===================>..........] - ETA: 5s - loss: 1226.3094 - acc: 0.1875 
Epoch 00002: val_acc did not improve from 0.25000
48/48 [==============================] - 17s 349ms/sample - loss: 859.3252 - acc: 0.2500 - val_loss: 98.1705 - val_acc: 0.1667
Epoch 3/8
32/48 [===================>..........] - ETA: 5s - loss: 58.3902 - acc: 0.3125 
Epoch 00003: val_acc improved from 0.25000 to 0.41667, saving model to model_rpsls.h5
48/48 [==============================] - 19s 392ms/sample - loss: 45.4646 - acc: 0.2917 - val_loss: 9.9707 - val_acc: 0.4167
Epoch 4/8
32/48 [===================>..........] - ETA: 5s - loss: 6.0821 - acc: 0

## Save our model

In [12]:
model.save("model_test.h5")